<b><center> Homework 7 </center></b>
<br>
<b><center> James Bonifield </center></b>
<br>
<br>
<b>Question 10.1</b><br>
Using the same crime data set as in Questions 8.2 and 9.1, find the best model you can using
(a) a regression tree model, and
(b) a random forest model.
In R, you can use the tree package or the rpart package, and the randomForest package. For
each model, describe one or two qualitative takeaways you get from analyzing the results (i.e., don’t just
stop when you have a good model, but interpret it too).

In [1]:
#First, Load Librarys:
library(tidyverse)
library(tree)
library(rpart)
library(rpart.plot)
library(ggplot2)
library(randomForest)
#Resize the plot dimensions so they come out better
options(repr.plot.width=10, repr.plot.height=5)

#Load Data
data <- suppressMessages((read_tsv('uscrime.txt')))

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



<b>Regression Tree Model</b><br>
First, we explore using a pruned and unpruned regression tree model. Let's get the unpruned model:

In [2]:
TreeModel <- tree(Crime~.,data=data)
summary(TreeModel)
predTree <- predict(TreeModel)
sseTree <- sum((predTree - data$Crime)^2)


Regression tree:
tree(formula = Crime ~ ., data = data)
Variables actually used in tree construction:
[1] "Po1" "Pop" "LF"  "NW" 
Number of terminal nodes:  7 
Residual mean deviance:  47390 = 1896000 / 40 
Distribution of residuals:
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-573.900  -98.300   -1.545    0.000  110.600  490.100 

Now we prune the tree down to the best 5 leaves. NB - this was an arbitrary choice of leaves just to explore the data a little more, no real stats thinking went into this choice :) 

In [3]:
PruneTree <- prune.tree(TreeModel, best=5)
PruneTreePred <- predict(PruneTree)
ssePruneTree <- sum((PruneTreePred - data$Crime)^2)
print(c("SSE for the Unpruned tree Model:", round(sseTree)))
print(c("SSE for the Pruned tree Model:  ", round(ssePruneTree)))

[1] "SSE for the Unpruned tree Model:" "1895722"                         
[1] "SSE for the Pruned tree Model:  " "2276670"                         


In [4]:
plot(TreeModel)
text(TreeModel, pos=3,cex=0.75)
plot(PruneTree)
text(PruneTree, pos=3,cex=0.75)

We see the SSE for the pruned model is much greater than the unpruned model - this makes sense, because the tree() function will fit the best possible tree with the minimum number of leaves that decrease the error, so pruning out any of those will increase the error of the model (though of course, the model could simply be overfitting the data, a perpetual concern...)

<b>Random Forest</b> <bR>
Now we create a random forest model from the crime data. First we'll compare a basic model with 500 vs 50,000 trees. As the data shows below, we see only a very marginal decrease in error, so 500 is probably a sufficiently large number of trees.

In [5]:
rfModel1 <- randomForest(Crime~., data=data, ntry = 5, ntree = 500, importance = TRUE)
rfModel1


Call:
 randomForest(formula = Crime ~ ., data = data, ntry = 5, ntree = 500,      importance = TRUE) 
               Type of random forest: regression
                     Number of trees: 500
No. of variables tried at each split: 5

          Mean of squared residuals: 84735.91
                    % Var explained: 42.12

In [6]:
rfModel2 <- randomForest(Crime~., data=data, ntry = 5, ntree = 50000, importance = TRUE)
rfModel2


Call:
 randomForest(formula = Crime ~ ., data = data, ntry = 5, ntree = 50000,      importance = TRUE) 
               Type of random forest: regression
                     Number of trees: 50000
No. of variables tried at each split: 5

          Mean of squared residuals: 85359.68
                    % Var explained: 41.7

In [7]:
predRF1 <- predict(rfModel1)
sseRF1 <- sum((predRF1 - data$Crime)^2)
round(sseRF1)
predRF2 <- predict(rfModel2)
sseRF2 <- sum((predRF2 - data$Crime)^2)
round(sseRF2)

[1] 3982588

[1] 4011905

We can see in the below diagram that the error dramatically decreases until around ~30 trees are hit, then it is fairly stable as the number of trees increases.

In [20]:
plot(rfModel1)

<b>Question 10.2</b><br>
Describe a situation or problem from your job, everyday life, current events, etc., for which a logistic
regression model would be appropriate. List some (up to 5) predictors that you might use.<br>

Logistic regression has a great deal of use cases, and would be an excellent tool to predict the original problem this class considered with the likelihood of a loan applicant to default on his or her loan. A different use case would be creating a logistic regression model to determine if a voter in the upcoming US midterm election would vote for a generic Republican or Democratic candidate. Some factors that might be considered would be:
- Age
- Income
- Who they voted for in the previous election
- Race/ethnicity
- Sex

<b>Question 10.3</b><br>
1. Using the GermanCredit data set, use logistic
regression to find a good predictive model for whether credit applicants are good credit risks or
not. Show your model (factors used and their coefficients), the software output, and the quality
of fit. You can use the glm function in R. To get a logistic regression (logit) model on data where 
the response is either zero or one, use family=binomial(link=”logit”) in your glm
function call.

2. Because the model gives a result between 0 and 1, it requires setting a threshold probability to
separate between “good” and “bad” answers. In this data set, they estimate that incorrectly
identifying a bad customer as good, is 5 times worse than incorrectly classifying a good
customer as bad. Determine a good threshold probability based on your model.


<b>Part 1: Build Logistic regression model</b>
<br> First, let's load the data and do some minor replacement:

In [21]:
#Load Data
GermanData <- (read.table('germancredit.txt', sep=' '))
#Replace 1=Good, 2=Bad with binomial 0/1 respectively

#GermanData$V21[GermanData$V21 == 1] = 0
GermanData$V21[GermanData$V21 == 2] = 0

head(GermanData)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,0
A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,0
A14,36,A32,A46,9055,A65,A73,2,A93,A101,...,A124,35,A143,A153,1,A172,2,A192,A201,1


Now, we split the data up into training and testing data (70-30 split)

In [22]:
RowCount <- nrow(GermanData)
GermanTrain <- GermanData[sample(1:RowCount,size = round(RowCount*0.70), replace = FALSE),]
GermanTest <- GermanData[-sample(1:RowCount,size = round(RowCount*0.70), replace = FALSE),]

In [23]:
glmModel1 <- glm(V21 ~. , family=binomial(link='logit'), data=GermanTrain)
summary(glmModel1)


Call:
glm(formula = V21 ~ ., family = binomial(link = "logit"), data = GermanTrain)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7062  -0.6081   0.3355   0.6598   2.4825  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.197e-01  1.381e+00  -0.087 0.930940    
V1A12        3.978e-01  2.720e-01   1.463 0.143522    
V1A13        1.828e+00  5.408e-01   3.380 0.000725 ***
V1A14        1.736e+00  2.882e-01   6.025  1.7e-09 ***
V2          -3.124e-02  1.159e-02  -2.695 0.007037 ** 
V3A31       -9.296e-01  7.270e-01  -1.279 0.200990    
V3A32        4.945e-01  5.174e-01   0.956 0.339222    
V3A33        8.036e-01  5.569e-01   1.443 0.149034    
V3A34        1.423e+00  5.264e-01   2.703 0.006879 ** 
V4A41        1.322e+00  4.532e-01   2.916 0.003540 ** 
V4A410       1.050e+00  8.953e-01   1.173 0.240725    
V4A42        8.279e-01  3.277e-01   2.527 0.011510 *  
V4A43        6.635e-01  3.083e-01   2.152 0.031372 *  
V4A44       -4.1

We'll select all signifcant variables (with $\alpha$ = 0.05) and rebuild the model

In [24]:
glmModel2 <- glm(V21 ~ V1+V2+V3+V4+V6+V7+V8+V14+V15, family=binomial(link='logit'), data=GermanTrain)
summary(glmModel2)


Call:
glm(formula = V21 ~ V1 + V2 + V3 + V4 + V6 + V7 + V8 + V14 + 
    V15, family = binomial(link = "logit"), data = GermanTrain)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.6008  -0.6735   0.3955   0.6916   2.3133  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -1.018800   0.745642  -1.366  0.17183    
V1A12         0.354067   0.254622   1.391  0.16436    
V1A13         1.635959   0.512812   3.190  0.00142 ** 
V1A14         1.597980   0.271451   5.887 3.94e-09 ***
V2           -0.045324   0.009088  -4.987 6.12e-07 ***
V3A31        -0.624296   0.663217  -0.941  0.34654    
V3A32         0.834598   0.472781   1.765  0.07751 .  
V3A33         0.798321   0.536596   1.488  0.13682    
V3A34         1.424606   0.499494   2.852  0.00434 ** 
V4A41         1.083146   0.408914   2.649  0.00808 ** 
V4A410        0.436260   0.826990   0.528  0.59783    
V4A42         0.640951   0.304856   2.102  0.03551 *  
V4A43         0.716152 

For this final iteration, we'll just drop off V6

In [25]:
glmModel3 <- glm(V21 ~ V1+V2+V4+V7+V8+V14+V15, family=binomial(link='logit'), data=GermanTrain)
summary(glmModel3)


Call:
glm(formula = V21 ~ V1 + V2 + V4 + V7 + V8 + V14 + V15, family = binomial(link = "logit"), 
    data = GermanTrain)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4468  -0.8237   0.4274   0.7148   1.9328  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.191489   0.574302  -0.333 0.738810    
V1A12         0.514140   0.238316   2.157 0.030975 *  
V1A13         1.730428   0.488592   3.542 0.000398 ***
V1A14         1.922402   0.257523   7.465 8.33e-14 ***
V2           -0.044428   0.008502  -5.226 1.74e-07 ***
V4A41         1.108535   0.395555   2.802 0.005071 ** 
V4A410        0.192334   0.768712   0.250 0.802430    
V4A42         0.554126   0.292432   1.895 0.058107 .  
V4A43         0.644157   0.277604   2.320 0.020318 *  
V4A44        -0.334116   0.943142  -0.354 0.723145    
V4A45        -0.378907   0.592241  -0.640 0.522313    
V4A46        -0.402651   0.463389  -0.869 0.384887    
V4A48        14.328193 530.193076

Now, we can rebuild all significant variables as binary:

In [26]:
GermanTrain$V1A13[GermanTrain$V1 == "A13"] <- 1
GermanTrain$V1A13[GermanTrain$V1 != "A13"] <- 0

GermanTrain$V2A41[GermanTrain$V2 == 'A41'] <- 1
GermanTrain$V2A41[GermanTrain$V2 != 'A41'] <- 0

GermanTrain$V4A41[GermanTrain$V4 == 'A41'] <- 1
GermanTrain$V4A41[GermanTrain$V4 != 'A41'] <- 0

GermanTrain$V7A74[GermanTrain$V7 == 'A74'] <- 1
GermanTrain$V7A74[GermanTrain$V7 != 'A74'] <- 0

GermanTrain$V14A142[GermanTrain$V14 == 'A142'] <- 1
GermanTrain$V14A142[GermanTrain$V14 != 'A142'] <- 0

GermanTrain$V15A152[GermanTrain$V4 == 'A152'] <- 1
GermanTrain$V15A152[GermanTrain$V4 != 'A152'] <- 0

GermanTest$V1A13[GermanTest$V1 == "A13"] <- 1
GermanTest$V1A13[GermanTest$V1 != "A13"] <- 0

GermanTest$V2A41[GermanTest$V2 == 'A41'] <- 1
GermanTest$V2A41[GermanTest$V2 != 'A41'] <- 0

GermanTest$V4A41[GermanTest$V4 == 'A41'] <- 1
GermanTest$V4A41[GermanTest$V4 != 'A41'] <- 0

GermanTest$V7A74[GermanTest$V7 == 'A74'] <- 1
GermanTest$V7A74[GermanTest$V7 != 'A74'] <- 0

GermanTest$V14A142[GermanTest$V14 == 'A142'] <- 1
GermanTest$V14A142[GermanTest$V14 != 'A142'] <- 0

GermanTest$V15A152[GermanTest$V4 == 'A152'] <- 1
GermanTest$V15A152[GermanTest$V4 != 'A152'] <- 0

In [27]:
ReplacedModel <-glm(V21 ~ V1+V2+V4+V7+V8+V14+V15, family=binomial(link='logit'), data=GermanTrain)
summary(ReplacedModel)


Call:
glm(formula = V21 ~ V1 + V2 + V4 + V7 + V8 + V14 + V15, family = binomial(link = "logit"), 
    data = GermanTrain)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4468  -0.8237   0.4274   0.7148   1.9328  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.191489   0.574302  -0.333 0.738810    
V1A12         0.514140   0.238316   2.157 0.030975 *  
V1A13         1.730428   0.488592   3.542 0.000398 ***
V1A14         1.922402   0.257523   7.465 8.33e-14 ***
V2           -0.044428   0.008502  -5.226 1.74e-07 ***
V4A41         1.108535   0.395555   2.802 0.005071 ** 
V4A410        0.192334   0.768712   0.250 0.802430    
V4A42         0.554126   0.292432   1.895 0.058107 .  
V4A43         0.644157   0.277604   2.320 0.020318 *  
V4A44        -0.334116   0.943142  -0.354 0.723145    
V4A45        -0.378907   0.592241  -0.640 0.522313    
V4A46        -0.402651   0.463389  -0.869 0.384887    
V4A48        14.328193 530.193076

In [28]:
LogisticPred <- predict(ReplacedModel,GermanTest, type= 'response')

In [29]:
#We'll analyze using the pROC package (receiver operating characteristic) that the TA uses in his video
library(pROC)
ROC <- roc(GermanTest$V21,LogisticPred)
plot(ROC)

<b>Part 2: Chosing a Threshold</b><br>
We can loop through 0-1 at incremements of 0.1 and minimize the loss, defined as $a+5b$, where a is identifying the good customer as bad and b is identifying a bad customer as good.
<br><b>Recall we made 1=Good and 0=Bad above</b>

In [30]:
Response <- c()
#Loop through, generate Confusion Matrix and calculate a+5b
for (i in 1:10) {
    BinaryPredicter <- as.integer(LogisticPred > i/10)
    tab <- table(BinaryPredicter,GermanTest$V21)
    Response[i]<- -999
    if (length(tab) == 4){
        a <- tab[2,1]
        b <- tab[1,2]
        Response[i] <- a+5*b
        }
    }
Response

[1]   87   97  106  135  142  226  321  457  695 -999

So, we see the best results at the lower end of our selection, at a threshold of 0.2. Let's drilldown to that range a little bit:

In [31]:
Response <- c()
#Loop through, generate Confusion Matrix and calculate a+5b
for (i in 1:20) {
    BinaryPredicter <- as.integer(LogisticPred > i/100)
    tab <- table(BinaryPredicter,GermanTest$V21)
    Response[i]<- -999
    if (length(tab) == 4){
        a <- tab[2,1]
        b <- tab[1,2]
        Response[i] <- a+5*b
        }
    }
Response

[1] -999 -999 -999 -999 -999 -999 -999 -999 -999   87   86   86   86   86   86
[16]   96   95   95   99   97

So, not really that much better, so we'll just stick with <b>0.2</b> as the best threshold for the logistic regression model we produced above.